<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch04/ch04-02-run_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

ON_COLAB: True
USE_GIT: True
fatal: destination path 'kagglebook' already exists and is not an empty directory.


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [4]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

In [5]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [6]:
# -----------------------------------
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

In [7]:
# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

In [8]:
# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

In [9]:
# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:03:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-logloss:0.41663	eval-logloss:0.43550
[1]	train-logloss:0.37126	eval-logloss:0.39889
[2]	train-logloss:0.33889	eval-logloss:0.37205
[3]	train-logloss:0.31320	eval-logloss:0.35606
[4]	train-logloss:0.29062	eval-logloss:0.33709
[5]	train-logloss:0.27315	eval-logloss:0.32549
[6]	train-logloss:0.25735	eval-logloss:0.31328
[7]	train-logloss:0.24299	eval-logloss:0.30434
[8]	train-logloss:0.22850	eval-logloss:0.29755
[9]	train-logloss:0.21984	eval-logloss:0.29490
[10]	train-logloss:0.21094	eval-logloss:0.28867
[11]	train-logloss:0.20280	eval-logloss:0.28290
[12]	train-logloss:0.19173	eval-logloss:0.27823
[13]	train-logloss:0.18626	eval-logloss:0.27372
[14]	train-logloss:0.17750	eval-logloss:0.27174
[15]	train-logloss:0.17111	eval-logloss:0.26724
[16]	train-logloss:0.16269	eval-logloss:0.26364
[17]	train-logloss:0.15749	eval-logloss:0.26171
[18]	train-logloss:0.15286	eval-logloss:0.25777
[19]	train-logloss:0.14977	eval-logloss:0.25564
[20]	train-logloss:0.14463	eval-logloss:0.25519
[2

In [10]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 0.2257


In [11]:
# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)

In [12]:
# -----------------------------------
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71,
          'eval_metric': 'logloss'}
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)

[0]	train-logloss:0.41663	eval-logloss:0.43550
[1]	train-logloss:0.37126	eval-logloss:0.39889


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:03:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[2]	train-logloss:0.33889	eval-logloss:0.37205
[3]	train-logloss:0.31320	eval-logloss:0.35606
[4]	train-logloss:0.29062	eval-logloss:0.33709
[5]	train-logloss:0.27315	eval-logloss:0.32549
[6]	train-logloss:0.25735	eval-logloss:0.31328
[7]	train-logloss:0.24299	eval-logloss:0.30434
[8]	train-logloss:0.22850	eval-logloss:0.29755
[9]	train-logloss:0.21984	eval-logloss:0.29490
[10]	train-logloss:0.21094	eval-logloss:0.28867
[11]	train-logloss:0.20280	eval-logloss:0.28290
[12]	train-logloss:0.19173	eval-logloss:0.27823
[13]	train-logloss:0.18626	eval-logloss:0.27372
[14]	train-logloss:0.17750	eval-logloss:0.27174
[15]	train-logloss:0.17111	eval-logloss:0.26724
[16]	train-logloss:0.16269	eval-logloss:0.26364
[17]	train-logloss:0.15749	eval-logloss:0.26171
[18]	train-logloss:0.15286	eval-logloss:0.25777
[19]	train-logloss:0.14977	eval-logloss:0.25564
[20]	train-logloss:0.14463	eval-logloss:0.25519
[21]	train-logloss:0.13899	eval-logloss:0.25306
[22]	train-logloss:0.13512	eval-logloss:0.25066


In [13]:
# 最適な決定木の本数で予測を行う
pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)

AttributeError: 'Booster' object has no attribute 'best_ntree_limit'